<a href="https://colab.research.google.com/github/waqqasansari/Machine_Learning_Probs/blob/master/hackerearth_novartis(CV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
train_df = pd.read_csv('Train.csv', dtype={'id': np.int32, 'target': np.int8})
Y = train_df['MULTIPLE_OFFENSE'].values
X = train_df.drop(['INCIDENT_ID','DATE', 'MULTIPLE_OFFENSE'], axis=1)
test_df = pd.read_csv('Test.csv', dtype={'id': np.int32})
test = test_df.drop(['INCIDENT_ID', 'DATE'], axis=1)

In [ ]:
print(X.isnull().values.any())

True


In [ ]:
X = X.fillna(X.mean())

In [ ]:
print(X.isnull().values.any())

False


In [ ]:
X

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36
1,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34
2,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34
3,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34
4,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23851,1,47,48,7,3,15,10,1,5,1,174,0.0,98,29,34
23852,0,33,32,2,1,5,6,0,5,1,174,0.0,112,29,43
23853,0,25,25,9,0,3,5,1,6,1,174,0.0,10,29,18
23854,0,39,39,6,5,2,7,1,6,1,127,0.0,112,103,43


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_features = scaler.transform(X)
X = pd.DataFrame(scaled_features, columns=X.columns)

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  2.3min finished



 Time taken: 0 hours 2 minutes and 28.06 seconds.


In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([16.03701081, 26.20842185, 23.22053971, 18.25293279, 19.98471522]), 'std_fit_time': array([0.1792395 , 0.62479131, 0.20065274, 0.12344481, 4.11639992]), 'mean_score_time': array([0.15283961, 0.25711317, 0.25310168, 0.18090696, 0.15414376]), 'std_score_time': array([0.0070998 , 0.0124805 , 0.06529151, 0.00576234, 0.0476522 ]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
         

In [ ]:
scaler.fit(test)
scaled_features = scaler.transform(test)
test = pd.DataFrame(scaled_features, columns=test.columns)

In [ ]:
y_test = random_search.predict(test)
results_df = pd.DataFrame(data={'INCIDENT_ID':test_df['INCIDENT_ID'], 'MULTIPLE_OFFENSE':y_test})
results_df.to_csv('submission(xgb4).csv', index=False)

In [ ]:
print(results_df)

      INCIDENT_ID  MULTIPLE_OFFENSE
0       CR_195453                 1
1       CR_103520                 1
2       CR_196089                 1
3       CR_112195                 1
4       CR_149832                 1
...           ...               ...
15898    CR_44468                 1
15899   CR_158460                 1
15900   CR_115946                 1
15901   CR_137663                 1
15902    CR_33545                 1

[15903 rows x 2 columns]
